In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor, plot_tree
from sklearn.tree import export_graphviz
import graphviz

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn import preprocessing

import gc
import xgboost as xgb
RANDOM_STATE=42

In [3]:
# Catgamos el dataset de train:
camps_file='Campanas_train.csv'
camps=pd.read_csv(camps_file)
camps.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
camps

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Resultado,Canal,Duracion_Campaña
0,1,A,A,A-A,2020-01-02,202001,0,B,1
1,1,B,B,B-B,2020-01-21,202001,0,C,1
2,1,A,A,A-A,2020-02-03,202002,0,B,1
3,1,B,B,B-B,2020-02-18,202002,0,C,1
4,1,A,A,A-A,2020-03-02,202003,0,B,1
...,...,...,...,...,...,...,...,...,...
493323,79539,B,B,B-B,2020-01-28,202001,0,C,1
493324,79539,C,D,C-D,2020-07-20,202007,0,C,1
493325,79539,A,A,A-A,2020-07-01,202007,0,B,1
493326,79539,B,B,B-B,2020-04-08,202004,0,C,1


In [24]:
camps.groupby(['id','Periodo','Producto-Tipo']).size()

id     Periodo     Producto-Tipo
1      2020-01-01  A-A              1
                   B-B              1
       2020-02-01  A-A              1
                   B-B              1
       2020-03-01  A-A              1
                                   ..
79539  2020-02-01  A-A              1
       2020-03-01  A-A              1
       2020-04-01  B-B              1
       2020-07-01  A-A              1
                   C-D              1
Length: 490521, dtype: int64

In [8]:
camps['Resultado'].value_counts(dropna=False)

0    483045
1     10283
Name: Resultado, dtype: int64

In [9]:
camps['Canal'].value_counts(dropna=False)

B    304948
C    188380
Name: Canal, dtype: int64

In [10]:
camps['Duracion_Campaña'].value_counts(dropna=False)

1     474397
2      18916
7         10
12         2
11         2
9          1
Name: Duracion_Campaña, dtype: int64

In [11]:
camps['Periodo'].value_counts(dropna=False)

202003    91923
202004    81288
202001    74762
202007    70285
202002    68419
202006    58071
202005    48580
Name: Periodo, dtype: int64

In [12]:
camps['Fecha_Campaña'].value_counts(dropna=False)

2020-03-02    52701
2020-06-01    35888
2020-01-02    35693
2020-07-01    35520
2020-02-03    35061
              ...  
2020-06-26       43
2020-05-01       33
2020-01-10       27
2020-01-06       25
2020-01-08       13
Name: Fecha_Campaña, Length: 153, dtype: int64

# Ponemos Peridodo como fecha

In [13]:
camps['Periodo']=pd.to_datetime(camps['Periodo'],format='%Y%m')

In [22]:
camps.describe(include='all')

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Resultado,Canal,Duracion_Campaña
count,493328.000000,493328,493328,493328,493328,493328,493328.000000,493328,493328.000000
unique,NaN,4,4,4,153,7,NaN,2,NaN
top,NaN,A,A,A-A,2020-03-02,2020-03-01 00:00:00,NaN,B,NaN
freq,NaN,286357,286357,286357,52701,91923,NaN,304948,NaN
first,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2020-07-01 00:00:00,NaN,NaN,NaN
mean,40198.652033,NaN,NaN,NaN,NaN,NaN,0.020844,NaN,1.038567
std,23881.997622,NaN,NaN,NaN,NaN,NaN,0.142863,NaN,0.196499
min,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000
25%,18968.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000


Tenemos estos peridods:
'2019-04-01', 
'2019-07-01',
'2019-10-01', 
'2020-01-01',
'2020-04-01', 
'2020-07-01',
'2020-10-01'

In [27]:
# Agrupamos por periodos :
# Vemos que como mucho hay una o dos campañas por cliente, mes ,tipo-producto
camps.groupby(['id','Periodo','Producto-Tipo']).size().reset_index(drop=False, name='Count')['Count'].value_counts()

1    487714
2      2807
Name: Count, dtype: int64

In [28]:
camps.groupby(['id','Periodo','Producto-Tipo']).size()

id     Periodo     Producto-Tipo
1      2020-01-01  A-A              1
                   B-B              1
       2020-02-01  A-A              1
                   B-B              1
       2020-03-01  A-A              1
                                   ..
79539  2020-02-01  A-A              1
       2020-03-01  A-A              1
       2020-04-01  B-B              1
       2020-07-01  A-A              1
                   C-D              1
Length: 490521, dtype: int64

In [29]:
camps

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Resultado,Canal,Duracion_Campaña
0,1,A,A,A-A,2020-01-02,2020-01-01,0,B,1
1,1,B,B,B-B,2020-01-21,2020-01-01,0,C,1
2,1,A,A,A-A,2020-02-03,2020-02-01,0,B,1
3,1,B,B,B-B,2020-02-18,2020-02-01,0,C,1
4,1,A,A,A-A,2020-03-02,2020-03-01,0,B,1
...,...,...,...,...,...,...,...,...,...
493323,79539,B,B,B-B,2020-01-28,2020-01-01,0,C,1
493324,79539,C,D,C-D,2020-07-20,2020-07-01,0,C,1
493325,79539,A,A,A-A,2020-07-01,2020-07-01,0,B,1
493326,79539,B,B,B-B,2020-04-08,2020-04-01,0,C,1


Tenemos estos peridods:
'2019-04-01', 
'2019-07-01',
'2019-10-01', 
'2020-01-01',
'2020-04-01', 
'2020-07-01',
'2020-10-01'

In [40]:
camps.drop('PeriodoT',axis=1,inplace=True)
camps.loc[ (camps['Periodo']==pd.to_datetime('202001',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('202001',format='%Y%m')

In [43]:
camps

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Resultado,Canal,Duracion_Campaña,PeriodoT
0,1,A,A,A-A,2020-01-02,2020-01-01,0,B,1,2020-01-01
1,1,B,B,B-B,2020-01-21,2020-01-01,0,C,1,2020-01-01
2,1,A,A,A-A,2020-02-03,2020-02-01,0,B,1,2020-04-01
3,1,B,B,B-B,2020-02-18,2020-02-01,0,C,1,2020-04-01
4,1,A,A,A-A,2020-03-02,2020-03-01,0,B,1,2020-04-01
...,...,...,...,...,...,...,...,...,...,...
493323,79539,B,B,B-B,2020-01-28,2020-01-01,0,C,1,2020-01-01
493324,79539,C,D,C-D,2020-07-20,2020-07-01,0,C,1,NaT
493325,79539,A,A,A-A,2020-07-01,2020-07-01,0,B,1,NaT
493326,79539,B,B,B-B,2020-04-08,2020-04-01,0,C,1,2020-04-01


In [42]:
camps.loc[ (camps['Periodo']==pd.to_datetime('202002',format='%Y%m')) |
           (camps['Periodo']==pd.to_datetime('202003',format='%Y%m')) |
           (camps['Periodo']==pd.to_datetime('202004',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('202004',format='%Y%m')

In [44]:
camps.loc[ (camps['Periodo']==pd.to_datetime('202005',format='%Y%m')) |
           (camps['Periodo']==pd.to_datetime('202006',format='%Y%m')) |
           (camps['Periodo']==pd.to_datetime('202007',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('202007',format='%Y%m')

In [45]:
camps

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Resultado,Canal,Duracion_Campaña,PeriodoT
0,1,A,A,A-A,2020-01-02,2020-01-01,0,B,1,2020-01-01
1,1,B,B,B-B,2020-01-21,2020-01-01,0,C,1,2020-01-01
2,1,A,A,A-A,2020-02-03,2020-02-01,0,B,1,2020-04-01
3,1,B,B,B-B,2020-02-18,2020-02-01,0,C,1,2020-04-01
4,1,A,A,A-A,2020-03-02,2020-03-01,0,B,1,2020-04-01
...,...,...,...,...,...,...,...,...,...,...
493323,79539,B,B,B-B,2020-01-28,2020-01-01,0,C,1,2020-01-01
493324,79539,C,D,C-D,2020-07-20,2020-07-01,0,C,1,2020-07-01
493325,79539,A,A,A-A,2020-07-01,2020-07-01,0,B,1,2020-07-01
493326,79539,B,B,B-B,2020-04-08,2020-04-01,0,C,1,2020-04-01


In [47]:
camps['PeriodoT'].value_counts(dropna=False)

2020-04-01    241630
2020-07-01    176936
2020-01-01     74762
Name: PeriodoT, dtype: int64

In [49]:
camps.groupby(['id','PeriodoT','Producto-Tipo']).size().reset_index(drop=False, name='Count')

,id,PeriodoT,Producto-Tipo,Count
0,1,2020-01-01,A-A,1
1,1,2020-01-01,B-B,1
2,1,2020-04-01,A-A,2
3,1,2020-04-01,B-B,2
4,1,2020-07-01,A-A,1
...,...,...,...,...
305144,79539,2020-01-01,B-B,1
305145,79539,2020-04-01,A-A,2
305146,79539,2020-04-01,B-B,1
305147,79539,2020-07-01,A-A,1


In [50]:
camps

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Resultado,Canal,Duracion_Campaña,PeriodoT
0,1,A,A,A-A,2020-01-02,2020-01-01,0,B,1,2020-01-01
1,1,B,B,B-B,2020-01-21,2020-01-01,0,C,1,2020-01-01
2,1,A,A,A-A,2020-02-03,2020-02-01,0,B,1,2020-04-01
3,1,B,B,B-B,2020-02-18,2020-02-01,0,C,1,2020-04-01
4,1,A,A,A-A,2020-03-02,2020-03-01,0,B,1,2020-04-01
...,...,...,...,...,...,...,...,...,...,...
493323,79539,B,B,B-B,2020-01-28,2020-01-01,0,C,1,2020-01-01
493324,79539,C,D,C-D,2020-07-20,2020-07-01,0,C,1,2020-07-01
493325,79539,A,A,A-A,2020-07-01,2020-07-01,0,B,1,2020-07-01
493326,79539,B,B,B-B,2020-04-08,2020-04-01,0,C,1,2020-04-01


In [51]:
# Antes de agrupar por periodos trimestrales, voy a hacer un OHE de las avriables categoricas. 
# ONEhotencoding en el producto
# ohe categoricas
_dummy_dataset = pd.get_dummies(camps['Canal'], prefix='Canal')
camps= pd.concat([camps,_dummy_dataset],axis=1)

In [52]:
camps

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Resultado,Canal,Duracion_Campaña,PeriodoT,Canal_B,Canal_C
0,1,A,A,A-A,2020-01-02,2020-01-01,0,B,1,2020-01-01,1,0
1,1,B,B,B-B,2020-01-21,2020-01-01,0,C,1,2020-01-01,0,1
2,1,A,A,A-A,2020-02-03,2020-02-01,0,B,1,2020-04-01,1,0
3,1,B,B,B-B,2020-02-18,2020-02-01,0,C,1,2020-04-01,0,1
4,1,A,A,A-A,2020-03-02,2020-03-01,0,B,1,2020-04-01,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
493323,79539,B,B,B-B,2020-01-28,2020-01-01,0,C,1,2020-01-01,0,1
493324,79539,C,D,C-D,2020-07-20,2020-07-01,0,C,1,2020-07-01,0,1
493325,79539,A,A,A-A,2020-07-01,2020-07-01,0,B,1,2020-07-01,1,0
493326,79539,B,B,B-B,2020-04-08,2020-04-01,0,C,1,2020-04-01,0,1


In [54]:
camps_=camps.pivot_table( index=['id','PeriodoT','Producto-Tipo'],
                         values=['Resultado','Duracion_Campaña','Canal_B','Canal_C'],
                         aggfunc=[len,np.sum])

In [57]:
camps_.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 305149 entries, (1, Timestamp('2020-01-01 00:00:00'), 'A-A') to (79539, Timestamp('2020-07-01 00:00:00'), 'C-D')
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   (len, Canal_B)           305149 non-null  uint8
 1   (len, Canal_C)           305149 non-null  uint8
 2   (len, Duracion_Campaña)  305149 non-null  int64
 3   (len, Resultado)         305149 non-null  int64
 4   (sum, Canal_B)           305149 non-null  uint8
 5   (sum, Canal_C)           305149 non-null  uint8
 6   (sum, Duracion_Campaña)  305149 non-null  int64
 7   (sum, Resultado)         305149 non-null  int64
dtypes: int64(4), uint8(4)
memory usage: 12.8+ MB


In [55]:
camps_

len                                     \
                               Canal_B Canal_C Duracion_Campaña Resultado   
id    PeriodoT   Producto-Tipo                                              
1     2020-01-01 A-A                 1       1                1         1   
                 B-B                 1       1                1         1   
      2020-04-01 A-A                 2       2                2         2   
                 B-B                 2       2                2         2   
      2020-07-01 A-A                 1       1                1         1   
...                                ...     ...              ...       ...   
79539 2020-01-01 B-B                 1       1                1         1   
      2020-04-01 A-A                 2       2                2         2   
                 B-B                 1       1                1         1   
      2020-07-01 A-A                 1       1                1         1   
                 C-D                 1       1                1         1   

                                   sum                                     
                               Canal_B Canal_C Duracion_Campaña Resultado  
id    PeriodoT   Producto-Tipo                                             
1     2020-01-01 A-A                 1       0                1         0  
                 B-B                 0       1                1         0  
      2020-04-01 A-A                 2       0                2         0  
                 B-B                 0       2                2         0  
      2020-07-01 A-A                 1       0                1         0  
...                                ...     ...              ...       ...  
79539 2020-01-01 B-B                 0       1                1         0  
      2020-04-01 A-A                 2       0                2         0  
                 B-B                 0       1                1         0  
      2020-07-01 A-A                 1       0                1         0  
                 C-D                 0       1                1         0  

[305149 rows x 8 columns]

In [58]:
camps_.columns=['Count','A','B','C','Canal_B','Canal_C','Duracion_campana','Resultado']

In [60]:
camps_.drop(['A','B','C'], axis=1, inplace=True)

In [62]:
camps_.reset_index(drop=False, inplace=True)

In [64]:
camps_.columns=['id','PeriodoT','Producto-Tipo','Count-campana','Canal_B','Canal_C','Duracion_campana','Resultado']

In [65]:
camps_

,id,PeriodoT,Producto-Tipo,Count-campana,Canal_B,Canal_C,Duracion_campana,Resultado
0,1,2020-01-01,A-A,1,1,0,1,0
1,1,2020-01-01,B-B,1,0,1,1,0
2,1,2020-04-01,A-A,2,2,0,2,0
3,1,2020-04-01,B-B,2,0,2,2,0
4,1,2020-07-01,A-A,1,1,0,1,0
...,...,...,...,...,...,...,...,...
305144,79539,2020-01-01,B-B,1,0,1,1,0
305145,79539,2020-04-01,A-A,2,2,0,2,0
305146,79539,2020-04-01,B-B,1,0,1,1,0
305147,79539,2020-07-01,A-A,1,1,0,1,0


# Producto cartesiano:

In [66]:
# producto cartesiano:
camps_['PeriodoT'].unique()

array(['2020-01-01T00:00:00.000000000', '2020-04-01T00:00:00.000000000',
       '2020-07-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [67]:
lista_fechas=['2020-01-01',
'2020-04-01', 
'2020-07-01',]

In [68]:
lista_clientes=camps_['id'].unique().tolist()

In [69]:
lista_clientes

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 101,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 174,
 175,
 176,
 177,
 178,
 179,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 189,
 190,
 191,
 192,
 19

In [70]:
camps_['Producto-Tipo'].value_counts(dropna=False)

A-A    148935
B-B     92305
C-D     41812
D-E     22097
Name: Producto-Tipo, dtype: int64

In [71]:
lista_productos=['A-A',
 'B-B',
 'C-D',
 'D-E']

In [72]:
len(lista_fechas), len(lista_clientes), len(lista_productos)

(3, 73745, 4)

In [73]:
print('Filas totales: '+ str(len(lista_fechas)*len(lista_clientes)*len(lista_productos)) )

Filas totales: 884940


In [74]:
cartesian_product = pd.MultiIndex.from_product([lista_clientes,lista_fechas, lista_productos ], 
                                               names = ['id','Periodo','Producto-Tipo'])
len(cartesian_product)

884940

In [75]:
cartesian_product

MultiIndex([(    1, '2020-01-01', 'A-A'),
            (    1, '2020-01-01', 'B-B'),
            (    1, '2020-01-01', 'C-D'),
            (    1, '2020-01-01', 'D-E'),
            (    1, '2020-04-01', 'A-A'),
            (    1, '2020-04-01', 'B-B'),
            (    1, '2020-04-01', 'C-D'),
            (    1, '2020-04-01', 'D-E'),
            (    1, '2020-07-01', 'A-A'),
            (    1, '2020-07-01', 'B-B'),
            ...
            (79539, '2020-01-01', 'C-D'),
            (79539, '2020-01-01', 'D-E'),
            (79539, '2020-04-01', 'A-A'),
            (79539, '2020-04-01', 'B-B'),
            (79539, '2020-04-01', 'C-D'),
            (79539, '2020-04-01', 'D-E'),
            (79539, '2020-07-01', 'A-A'),
            (79539, '2020-07-01', 'B-B'),
            (79539, '2020-07-01', 'C-D'),
            (79539, '2020-07-01', 'D-E')],
           names=['id', 'Periodo', 'Producto-Tipo'], length=884940)

In [76]:
full_df_1 = pd.DataFrame(index = cartesian_product).reset_index()
full_df_1.tail()

,id,Periodo,Producto-Tipo
884935,79539,2020-04-01,D-E
884936,79539,2020-07-01,A-A
884937,79539,2020-07-01,B-B
884938,79539,2020-07-01,C-D
884939,79539,2020-07-01,D-E


In [77]:
full_df_1['Periodo']=pd.to_datetime(full_df_1['Periodo'], format='%Y-%m-%d')

In [78]:
full_df_1

,id,Periodo,Producto-Tipo
0,1,2020-01-01,A-A
1,1,2020-01-01,B-B
2,1,2020-01-01,C-D
3,1,2020-01-01,D-E
4,1,2020-04-01,A-A
...,...,...,...
884935,79539,2020-04-01,D-E
884936,79539,2020-07-01,A-A
884937,79539,2020-07-01,B-B
884938,79539,2020-07-01,C-D


In [79]:
full_df_1.groupby(['id','Periodo']).agg({'Producto-Tipo':len})

Producto-Tipo
id    Periodo                  
1     2020-01-01              4
      2020-04-01              4
      2020-07-01              4
2     2020-01-01              4
      2020-04-01              4
...                         ...
79538 2020-04-01              4
      2020-07-01              4
79539 2020-01-01              4
      2020-04-01              4
      2020-07-01              4

[221235 rows x 1 columns]

In [82]:
full_df_1.columns=['id','PeriodoT','Producto-Tipo']

# Hacemos el Merge en Train

In [84]:
full_df_1 = pd.merge(full_df_1,camps_ , on = ['id','PeriodoT','Producto-Tipo'], how = 'left')

In [85]:
full_df_1

,id,PeriodoT,Producto-Tipo,Count-campana,Canal_B,Canal_C,Duracion_campana,Resultado
0,1,2020-01-01,A-A,1.0,1.0,0.0,1.0,0.0
1,1,2020-01-01,B-B,1.0,0.0,1.0,1.0,0.0
2,1,2020-01-01,C-D,NaN,NaN,NaN,NaN,NaN
3,1,2020-01-01,D-E,NaN,NaN,NaN,NaN,NaN
4,1,2020-04-01,A-A,2.0,2.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...
884935,79539,2020-04-01,D-E,NaN,NaN,NaN,NaN,NaN
884936,79539,2020-07-01,A-A,1.0,1.0,0.0,1.0,0.0
884937,79539,2020-07-01,B-B,NaN,NaN,NaN,NaN,NaN
884938,79539,2020-07-01,C-D,1.0,0.0,1.0,1.0,0.0


In [86]:
full_df_1.groupby(['id','PeriodoT'])['Producto-Tipo'].size()

id     PeriodoT  
1      2020-01-01    4
       2020-04-01    4
       2020-07-01    4
2      2020-01-01    4
       2020-04-01    4
                    ..
79538  2020-04-01    4
       2020-07-01    4
79539  2020-01-01    4
       2020-04-01    4
       2020-07-01    4
Name: Producto-Tipo, Length: 221235, dtype: int64

In [87]:
# Pero por el producto cartesiano se han generado un monton de nulos
full_df_1.isnull().sum()

id                       0
PeriodoT                 0
Producto-Tipo            0
Count-campana       579791
Canal_B             579791
Canal_C             579791
Duracion_campana    579791
Resultado           579791
dtype: int64

In [88]:
# Rellenamos con ceros
full_df_1.fillna(0,inplace=True)

In [89]:
full_df_1

,id,PeriodoT,Producto-Tipo,Count-campana,Canal_B,Canal_C,Duracion_campana,Resultado
0,1,2020-01-01,A-A,1.0,1.0,0.0,1.0,0.0
1,1,2020-01-01,B-B,1.0,0.0,1.0,1.0,0.0
2,1,2020-01-01,C-D,0.0,0.0,0.0,0.0,0.0
3,1,2020-01-01,D-E,0.0,0.0,0.0,0.0,0.0
4,1,2020-04-01,A-A,2.0,2.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...
884935,79539,2020-04-01,D-E,0.0,0.0,0.0,0.0,0.0
884936,79539,2020-07-01,A-A,1.0,1.0,0.0,1.0,0.0
884937,79539,2020-07-01,B-B,0.0,0.0,0.0,0.0,0.0
884938,79539,2020-07-01,C-D,1.0,0.0,1.0,1.0,0.0


In [90]:
# Hacemos un pickel en este punto
full_df_1.to_pickle('./3T/fulldf_base_campanas_train.pkl',compression='zip')


# Hacemos lo mismo con Test

In [96]:
# Catgamos el dataset de train:
camps_file='Campanas_test.csv'
camps_test=pd.read_csv(camps_file)
camps_test.drop('Unnamed: 0',axis=1,inplace=True)

In [92]:
camps_test

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Canal,Duracion_Campaña
0,1,A,A,A-A,2020-09-03,202009,B,1
1,1,D,E,D-E,2020-10-23,202010,C,1
2,1,B,B,B-B,2020-08-27,202008,C,1
3,2,A,A,A-A,2020-09-03,202009,B,1
4,2,C,D,C-D,2020-09-02,202009,C,1
...,...,...,...,...,...,...,...,...
225993,79538,A,A,A-A,2020-10-02,202010,B,1
225994,79539,C,D,C-D,2020-10-26,202010,C,2
225995,79539,A,A,A-A,2020-10-02,202010,B,1
225996,79539,D,E,D-E,2020-10-23,202010,C,1


In [97]:
camps_test['Periodo']=pd.to_datetime(camps_test['Periodo'],format='%Y%m')

In [98]:
camps_test

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Canal,Duracion_Campaña
0,1,A,A,A-A,2020-09-03,2020-09-01,B,1
1,1,D,E,D-E,2020-10-23,2020-10-01,C,1
2,1,B,B,B-B,2020-08-27,2020-08-01,C,1
3,2,A,A,A-A,2020-09-03,2020-09-01,B,1
4,2,C,D,C-D,2020-09-02,2020-09-01,C,1
...,...,...,...,...,...,...,...,...
225993,79538,A,A,A-A,2020-10-02,2020-10-01,B,1
225994,79539,C,D,C-D,2020-10-26,2020-10-01,C,2
225995,79539,A,A,A-A,2020-10-02,2020-10-01,B,1
225996,79539,D,E,D-E,2020-10-23,2020-10-01,C,1


In [99]:
camps_test.loc[ (camps_test['Periodo']==pd.to_datetime('202008',format='%Y%m')) |
           (camps_test['Periodo']==pd.to_datetime('202009',format='%Y%m')) |
           (camps_test['Periodo']==pd.to_datetime('202010',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('202010',format='%Y%m')

In [100]:
camps_test

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Canal,Duracion_Campaña,PeriodoT
0,1,A,A,A-A,2020-09-03,2020-09-01,B,1,2020-10-01
1,1,D,E,D-E,2020-10-23,2020-10-01,C,1,2020-10-01
2,1,B,B,B-B,2020-08-27,2020-08-01,C,1,2020-10-01
3,2,A,A,A-A,2020-09-03,2020-09-01,B,1,2020-10-01
4,2,C,D,C-D,2020-09-02,2020-09-01,C,1,2020-10-01
...,...,...,...,...,...,...,...,...,...
225993,79538,A,A,A-A,2020-10-02,2020-10-01,B,1,2020-10-01
225994,79539,C,D,C-D,2020-10-26,2020-10-01,C,2,2020-10-01
225995,79539,A,A,A-A,2020-10-02,2020-10-01,B,1,2020-10-01
225996,79539,D,E,D-E,2020-10-23,2020-10-01,C,1,2020-10-01


In [101]:
# Antes de agrupar por periodos trimestrales, voy a hacer un OHE de las avriables categoricas. 
# ONEhotencoding en el producto
# ohe categoricas
_dummy_dataset = pd.get_dummies(camps_test['Canal'], prefix='Canal')
camps_test= pd.concat([camps_test,_dummy_dataset],axis=1)

In [102]:
camps_test

,id,Id_Producto,Tipo,Producto-Tipo,Fecha_Campaña,Periodo,Canal,Duracion_Campaña,PeriodoT,Canal_B,Canal_C
0,1,A,A,A-A,2020-09-03,2020-09-01,B,1,2020-10-01,1,0
1,1,D,E,D-E,2020-10-23,2020-10-01,C,1,2020-10-01,0,1
2,1,B,B,B-B,2020-08-27,2020-08-01,C,1,2020-10-01,0,1
3,2,A,A,A-A,2020-09-03,2020-09-01,B,1,2020-10-01,1,0
4,2,C,D,C-D,2020-09-02,2020-09-01,C,1,2020-10-01,0,1
...,...,...,...,...,...,...,...,...,...,...,...
225993,79538,A,A,A-A,2020-10-02,2020-10-01,B,1,2020-10-01,1,0
225994,79539,C,D,C-D,2020-10-26,2020-10-01,C,2,2020-10-01,0,1
225995,79539,A,A,A-A,2020-10-02,2020-10-01,B,1,2020-10-01,1,0
225996,79539,D,E,D-E,2020-10-23,2020-10-01,C,1,2020-10-01,0,1


In [105]:
camps_test_=camps_test.pivot_table( index=['id','PeriodoT','Producto-Tipo'],
                         values=['Duracion_Campaña','Canal_B','Canal_C'],
                         aggfunc=[len,np.sum])

In [106]:
camps_test_

len                              sum  \
                               Canal_B Canal_C Duracion_Campaña Canal_B   
id    PeriodoT   Producto-Tipo                                            
1     2020-10-01 A-A                 1       1                1       1   
                 B-B                 1       1                1       0   
                 D-E                 1       1                1       0   
2     2020-10-01 A-A                 1       1                1       1   
                 B-B                 1       1                1       0   
...                                ...     ...              ...     ...   
79537 2020-10-01 C-D                 1       1                1       0   
79538 2020-10-01 A-A                 3       3                3       3   
79539 2020-10-01 A-A                 2       2                2       2   
                 C-D                 1       1                1       0   
                 D-E                 1       1                1       0   

                                                         
                               Canal_C Duracion_Campaña  
id    PeriodoT   Producto-Tipo                           
1     2020-10-01 A-A                 0                1  
                 B-B                 1                1  
                 D-E                 1                1  
2     2020-10-01 A-A                 0                1  
                 B-B                 1                1  
...                                ...              ...  
79537 2020-10-01 C-D                 1                2  
79538 2020-10-01 A-A                 0                3  
79539 2020-10-01 A-A                 0                2  
                 C-D                 1                2  
                 D-E                 1                1  

[107676 rows x 6 columns]

In [107]:
camps_test_.columns=['Count','A','B','Canal_B','Canal_C','Duracion_campana']

In [108]:
camps_test_

Count  A  B  Canal_B  Canal_C  \
id    PeriodoT   Producto-Tipo                                  
1     2020-10-01 A-A                1  1  1        1        0   
                 B-B                1  1  1        0        1   
                 D-E                1  1  1        0        1   
2     2020-10-01 A-A                1  1  1        1        0   
                 B-B                1  1  1        0        1   
...                               ... .. ..      ...      ...   
79537 2020-10-01 C-D                1  1  1        0        1   
79538 2020-10-01 A-A                3  3  3        3        0   
79539 2020-10-01 A-A                2  2  2        2        0   
                 C-D                1  1  1        0        1   
                 D-E                1  1  1        0        1   

                                Duracion_campana  
id    PeriodoT   Producto-Tipo                    
1     2020-10-01 A-A                           1  
                 B-B                           1  
                 D-E                           1  
2     2020-10-01 A-A                           1  
                 B-B                           1  
...                                          ...  
79537 2020-10-01 C-D                           2  
79538 2020-10-01 A-A                           3  
79539 2020-10-01 A-A                           2  
                 C-D                           2  
                 D-E                           1  

[107676 rows x 6 columns]

In [109]:
# Añadimos Resultado a 0
camps_test_['Resultado']=0

In [110]:
camps_test_

Count  A  B  Canal_B  Canal_C  \
id    PeriodoT   Producto-Tipo                                  
1     2020-10-01 A-A                1  1  1        1        0   
                 B-B                1  1  1        0        1   
                 D-E                1  1  1        0        1   
2     2020-10-01 A-A                1  1  1        1        0   
                 B-B                1  1  1        0        1   
...                               ... .. ..      ...      ...   
79537 2020-10-01 C-D                1  1  1        0        1   
79538 2020-10-01 A-A                3  3  3        3        0   
79539 2020-10-01 A-A                2  2  2        2        0   
                 C-D                1  1  1        0        1   
                 D-E                1  1  1        0        1   

                                Duracion_campana  Resultado  
id    PeriodoT   Producto-Tipo                               
1     2020-10-01 A-A                           1          0  
                 B-B                           1          0  
                 D-E                           1          0  
2     2020-10-01 A-A                           1          0  
                 B-B                           1          0  
...                                          ...        ...  
79537 2020-10-01 C-D                           2          0  
79538 2020-10-01 A-A                           3          0  
79539 2020-10-01 A-A                           2          0  
                 C-D                           2          0  
                 D-E                           1          0  

[107676 rows x 7 columns]

In [111]:
camps_test_.drop(['A','B'], axis=1, inplace=True)

In [112]:
camps_test_.reset_index(drop=False, inplace=True)

In [115]:
camps_test_

,id,PeriodoT,Producto-Tipo,Count-campana,Canal_B,Canal_C,Duracion_campana,Resultado
0,1,2020-10-01,A-A,1,1,0,1,0
1,1,2020-10-01,B-B,1,0,1,1,0
2,1,2020-10-01,D-E,1,0,1,1,0
3,2,2020-10-01,A-A,1,1,0,1,0
4,2,2020-10-01,B-B,1,0,1,1,0
...,...,...,...,...,...,...,...,...
107671,79537,2020-10-01,C-D,1,0,1,2,0
107672,79538,2020-10-01,A-A,3,3,0,3,0
107673,79539,2020-10-01,A-A,2,2,0,2,0
107674,79539,2020-10-01,C-D,1,0,1,2,0


In [114]:
camps_test_.columns=['id','PeriodoT','Producto-Tipo','Count-campana','Canal_B','Canal_C','Duracion_campana','Resultado']

In [ ]:
cartesian_product = pd.MultiIndex.from_product([lista_clientes,lista_fechas, lista_productos ], 
                                               names = ['id','Periodo','Producto-Tipo'])
len(cartesian_product)

cartesian_product

full_df = pd.DataFrame(index = cartesian_product).reset_index()
full_df.tail()

full_df['Periodo']=pd.to_datetime(full_df['Periodo'], format='%Y-%m-%d')

full_df

full_df.groupby(['id','Periodo']).agg({'Producto-Tipo':len})

full_df.columns=['id','PeriodoT','Producto-Tipo']


MERGE

full_df = pd.merge(full_df,camps_test_ , on = ['id','PeriodoT','Producto-Tipo'], how = 'left')

full_df

full_df.groupby(['id','PeriodoT'])['Producto-Tipo'].size()

# Pero por el producto cartesiano se han generado un monton de nulos
full_df.isnull().sum()

# Rellenamos con ceros
full_df.fillna(0,inplace=True)